In [1]:
import warnings

warnings.simplefilter(action="ignore", category=FutureWarning)
import pandas as pd  # noqa: E402

In [2]:
def get_results(dataset, disc_model, method, columns):
    print(dataset, disc_model, method)

    df = pd.DataFrame(columns=columns)

    for fold_n in range(5):
        try:
            metrics_path = f"../models/{dataset}/{method}/fold_{fold_n}/cf_metrics_{disc_model}.csv"
            df_ = pd.read_csv(metrics_path)
            df = pd.concat([df, df_], axis=0)
        except Exception as e:
            pass
            # print(f"File not found: {metrics_path}")
            # df_ = pd.DataFrame(columns=columns)
            # df = pd.concat([df, df_], axis=0)
    # print(df.shape)
    df["dataset"] = dataset
    df["disc_model"] = disc_model
    df["method"] = method

    return df

In [3]:
columns = [
    "dataset",
    "disc_model",
    "method",
    "K_vectors",
    "validity",
    "prob_plausibility",
    "cf_belongs_to_group",
    "log_density_cf",
    "proximity_continuous_manhattan",
    "proximity_continuous_euclidean",
    "isolation_forest_scores_cf",
    "lof_scores_cf",
    "time",
]
datasets = [
    "BlobsDataset",
    "LawDataset",
    "MoonsDataset",
    "WineDataset",
    "HelocDataset",
    "DigitsDataset",
]
disc_models = ["MultinomialLogisticRegression", "MultilayerPerceptron"]
global_methods = ["AReS", "GLOBE_CE", "RPPCEF_GLOBAL"]  # "GCE"
local_methods = ["wach", "Artelt", "RPPCEF_LOCAL"]  # "PPCEF_2"
group_methods = ["RPPCEF"]  # "GLANCE",
methods = global_methods + local_methods + group_methods

dataset = datasets[5]
disc_model = disc_models[0]
# method = local_methods[2]

df_all = pd.DataFrame(columns=columns)
for disc_model in disc_models:
    for dataset in datasets:
        for method in global_methods:
            df = get_results(dataset, disc_model, method, columns)
            df_all = pd.concat([df_all, df], axis=0)

BlobsDataset MultinomialLogisticRegression AReS
BlobsDataset MultinomialLogisticRegression GLOBE_CE
BlobsDataset MultinomialLogisticRegression RPPCEF_GLOBAL
LawDataset MultinomialLogisticRegression AReS
LawDataset MultinomialLogisticRegression GLOBE_CE
LawDataset MultinomialLogisticRegression RPPCEF_GLOBAL
MoonsDataset MultinomialLogisticRegression AReS
MoonsDataset MultinomialLogisticRegression GLOBE_CE
MoonsDataset MultinomialLogisticRegression RPPCEF_GLOBAL
WineDataset MultinomialLogisticRegression AReS
WineDataset MultinomialLogisticRegression GLOBE_CE
WineDataset MultinomialLogisticRegression RPPCEF_GLOBAL
HelocDataset MultinomialLogisticRegression AReS
HelocDataset MultinomialLogisticRegression GLOBE_CE
HelocDataset MultinomialLogisticRegression RPPCEF_GLOBAL
DigitsDataset MultinomialLogisticRegression AReS
DigitsDataset MultinomialLogisticRegression GLOBE_CE
DigitsDataset MultinomialLogisticRegression RPPCEF_GLOBAL
BlobsDataset MultilayerPerceptron AReS
BlobsDataset MultilayerPe

In [4]:
df_all

,dataset,disc_model,method,K_vectors,validity,prob_plausibility,cf_belongs_to_group,log_density_cf,proximity_continuous_manhattan,proximity_continuous_euclidean,...,actionability,sparsity,proximity_categorical_hamming,proximity_categorical_jaccard,proximity_continuous_mad,proximity_l2_jaccard,proximity_mad_hamming,log_density_test,lof_scores_test,isolation_forest_scores_test
0,BlobsDataset,MultinomialLogisticRegression,GLOBE_CE,NaN,1.000000,0.000000,NaN,-6.394403,0.277536,0.266591,...,0.0,1.0,NaN,0.266591,1.275073,0.266591,NaN,2.541622,1.120480,0.017491
0,BlobsDataset,MultinomialLogisticRegression,GLOBE_CE,NaN,1.000000,0.000000,NaN,-6.702945,0.305860,0.304525,...,0.0,1.0,NaN,0.304525,1.380082,0.304525,NaN,2.716548,1.063162,0.026827
0,BlobsDataset,MultinomialLogisticRegression,GLOBE_CE,NaN,1.000000,0.000000,NaN,-4.963414,0.287677,0.277575,...,0.0,1.0,NaN,0.277575,1.330172,0.277575,NaN,2.604315,1.110090,0.022112
0,BlobsDataset,MultinomialLogisticRegression,GLOBE_CE,NaN,1.000000,0.000000,NaN,-6.062356,0.299691,0.298250,...,0.0,1.0,NaN,0.298250,1.398332,0.298250,NaN,2.561564,1.112318,0.017147
0,BlobsDataset,MultinomialLogisticRegression,RPPCEF_GLOBAL,1,1.000000,0.890000,1.0,2.632997,0.729519,0.517640,...,0.0,1.0,NaN,0.517640,2.987973,0.517640,NaN,2.479431,1.100914,0.025462
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,DigitsDataset,MultilayerPerceptron,RPPCEF_GLOBAL,1,1.000000,0.819820,1.0,-98.922220,89.205008,17.369059,...,0.0,1.0,NaN,17.369059,60.185738,17.369059,NaN,-93.829170,1.080309,0.092435
0,DigitsDataset,MultilayerPerceptron,RPPCEF_GLOBAL,1,1.000000,0.765766,1.0,-99.275860,87.649704,16.415904,...,0.0,1.0,NaN,16.415904,59.080132,16.415904,NaN,-93.214710,1.082217,0.088308
0,DigitsDataset,MultilayerPerceptron,RPPCEF_GLOBAL,1,1.000000,0.714286,1.0,-99.691790,87.925648,16.593116,...,0.0,1.0,NaN,16.593116,59.018745,16.593116,NaN,-94.007835,1.083431,0.083264
0,DigitsDataset,MultilayerPerceptron,RPPCEF_GLOBAL,1,1.000000,0.684685,1.0,-98.860390,91.022525,17.524019,...,0.0,1.0,NaN,17.524019,58.798030,17.524019,NaN,-92.326130,1.072227,0.085885


In [5]:
df_all.to_csv("results_GLOBAL.csv", index=False)

In [6]:
df_all.groupby(["disc_model", "dataset", "method"]).mean().round(2)

K_vectors  validity  \
disc_model                    dataset       method                              
MultilayerPerceptron          BlobsDataset  GLOBE_CE            NaN      0.99   
                                            RPPCEF_GLOBAL       1.0      1.00   
                              DigitsDataset GLOBE_CE            NaN      0.00   
                                            RPPCEF_GLOBAL       1.0      1.00   
                              HelocDataset  AReS                NaN      0.28   
                                            GLOBE_CE            NaN      1.00   
                                            RPPCEF_GLOBAL       1.0      1.00   
                              LawDataset    GLOBE_CE            NaN      1.00   
                                            RPPCEF_GLOBAL       1.0      1.00   
                              MoonsDataset  GLOBE_CE            NaN      1.00   
                                            RPPCEF_GLOBAL       1.0      0.91   
                              WineDataset   GLOBE_CE            NaN      1.00   
                                            RPPCEF_GLOBAL       1.0      1.00   
MultinomialLogisticRegression BlobsDataset  GLOBE_CE            NaN      1.00   
                                            RPPCEF_GLOBAL       1.0      1.00   
                              DigitsDataset GLOBE_CE            NaN      0.00   
                                            RPPCEF_GLOBAL       1.0      1.00   
                              HelocDataset  AReS                NaN      0.18   
                                            GLOBE_CE            NaN      1.00   
                                            RPPCEF_GLOBAL       1.0      1.00   
                              LawDataset    GLOBE_CE            NaN      1.00   
                                            RPPCEF_GLOBAL       1.0      1.00   
                              MoonsDataset  GLOBE_CE            NaN      1.00   
                                            RPPCEF_GLOBAL       1.0      1.00   
                              WineDataset   GLOBE_CE            NaN      1.00   
                                            RPPCEF_GLOBAL       1.0      1.00   

                                                           prob_plausibility  \
disc_model                    dataset       method                             
MultilayerPerceptron          BlobsDataset  GLOBE_CE                    0.00   
                                            RPPCEF_GLOBAL               0.92   
                              DigitsDataset GLOBE_CE                     NaN   
                                            RPPCEF_GLOBAL               0.72   
                              HelocDataset  AReS                        0.18   
                                            GLOBE_CE                    0.17   
                                            RPPCEF_GLOBAL               0.46   
                              LawDataset    GLOBE_CE                    0.37   
                                            RPPCEF_GLOBAL               0.79   
                              MoonsDataset  GLOBE_CE                    0.00   
                                            RPPCEF_GLOBAL               0.63   
                              WineDataset   GLOBE_CE                    0.00   
                                            RPPCEF_GLOBAL               0.95   
MultinomialLogisticRegression BlobsDataset  GLOBE_CE                    0.00   
                                            RPPCEF_GLOBAL               0.92   
                              DigitsDataset GLOBE_CE                    0.00   
                                            RPPCEF_GLOBAL               0.69   
                              HelocDataset  AReS                        0.07   
                                            GLOBE_CE                    0.13   
                                            RPPCEF_GLOBAL               0.46   
                             

In [7]:
columns = [
    "dataset",
    "disc_model",
    "method",
    "K_vectors",
    "validity",
    "prob_plausibility",
    "cf_belongs_to_group",
    "log_density_cf",
    "proximity_continuous_manhattan",
    "proximity_continuous_euclidean",
    "isolation_forest_scores_cf",
    "lof_scores_cf",
    "time",
]
datasets = [
    "BlobsDataset",
    "LawDataset",
    "MoonsDataset",
    "WineDataset",
    "HelocDataset",
    "DigitsDataset",
]
disc_models = ["MultinomialLogisticRegression", "MultilayerPerceptron"]
global_methods = ["AReS", "GLOBE_CE", "RPPCEF_GLOBAL"]  # "GCE"
local_methods = ["wach", "Artelt", "RPPCEF_LOCAL"]  # "PPCEF_2"
group_methods = ["RPPCEF"]  # "GLANCE",
methods = global_methods + local_methods + group_methods

dataset = datasets[5]
disc_model = disc_models[0]
# method = local_methods[2]

df_all = pd.DataFrame(columns=columns)
for disc_model in disc_models:
    for dataset in datasets:
        for method in local_methods:
            df = get_results(dataset, disc_model, method, columns)
            df_all = pd.concat([df_all, df], axis=0)

BlobsDataset MultinomialLogisticRegression wach
BlobsDataset MultinomialLogisticRegression Artelt
BlobsDataset MultinomialLogisticRegression RPPCEF_LOCAL
LawDataset MultinomialLogisticRegression wach
LawDataset MultinomialLogisticRegression Artelt
LawDataset MultinomialLogisticRegression RPPCEF_LOCAL
MoonsDataset MultinomialLogisticRegression wach
MoonsDataset MultinomialLogisticRegression Artelt
MoonsDataset MultinomialLogisticRegression RPPCEF_LOCAL
WineDataset MultinomialLogisticRegression wach
WineDataset MultinomialLogisticRegression Artelt
WineDataset MultinomialLogisticRegression RPPCEF_LOCAL
HelocDataset MultinomialLogisticRegression wach
HelocDataset MultinomialLogisticRegression Artelt
HelocDataset MultinomialLogisticRegression RPPCEF_LOCAL
DigitsDataset MultinomialLogisticRegression wach
DigitsDataset MultinomialLogisticRegression Artelt
DigitsDataset MultinomialLogisticRegression RPPCEF_LOCAL
BlobsDataset MultilayerPerceptron wach
BlobsDataset MultilayerPerceptron Artelt
Bl

In [9]:
df_all.to_csv("results_LOCAL.csv", index=False)

In [10]:
df_all.groupby(["disc_model", "dataset", "method"]).mean().round(2)

K_vectors  validity  \
disc_model                    dataset       method                             
MultilayerPerceptron          BlobsDataset  RPPCEF_LOCAL     100.2      1.00   
                                            wach               NaN      0.77   
                              DigitsDataset RPPCEF_LOCAL     110.4      1.00   
                              HelocDataset  RPPCEF_LOCAL    1043.4      1.00   
                                            wach               NaN      1.00   
                              LawDataset    RPPCEF_LOCAL     219.2      1.00   
                                            wach               NaN      1.00   
                              MoonsDataset  RPPCEF_LOCAL     102.4      1.00   
                                            wach               NaN      1.00   
                              WineDataset   RPPCEF_LOCAL      11.6      1.00   
                                            wach               NaN      0.92   
MultinomialLogisticRegression BlobsDataset  Artelt             NaN      1.00   
                                            RPPCEF_LOCAL     100.4      1.00   
                                            wach               NaN      0.90   
                              DigitsDataset Artelt             NaN      1.00   
                                            RPPCEF_LOCAL     110.8      1.00   
                                            wach               NaN      0.00   
                              HelocDataset  RPPCEF_LOCAL    1037.2      1.00   
                                            wach               NaN      1.00   
                              LawDataset    Artelt             NaN      1.00   
                                            RPPCEF_LOCAL     220.8      1.00   
                                            wach               NaN      1.00   
                              MoonsDataset  Artelt             NaN      1.00   
                                            RPPCEF_LOCAL     102.6      1.00   
                                            wach               NaN      1.00   
                              WineDataset   Artelt             NaN      1.00   
                                            RPPCEF_LOCAL      11.6      1.00   
                                            wach               NaN      1.00   

                                                          prob_plausibility  \
disc_model                    dataset       method                            
MultilayerPerceptron          BlobsDataset  RPPCEF_LOCAL               1.00   
                                            wach                       0.00   
                              DigitsDataset RPPCEF_LOCAL               1.00   
                              HelocDataset  RPPCEF_LOCAL               1.00   
                                            wach                       0.60   
                              LawDataset    RPPCEF_LOCAL               1.00   
                                            wach                       0.45   
                              MoonsDataset  RPPCEF_LOCAL               1.00   
                                            wach                       0.00   
                              WineDataset   RPPCEF_LOCAL               1.00   
                                            wach                       0.08   
MultinomialLogisticRegression BlobsDataset  Artelt                     0.00   
                                            RPPCEF_LOCAL               1.00   
                                            wach                       0.00   
                              DigitsDataset Artelt                     0.00   
                                            RPPCEF_LOCAL               1.00   
                                            wach                        NaN   
                              HelocDataset  RPPCEF_LOCAL               1.00   
                                            wach                       0.00 

In [11]:
columns = [
    "dataset",
    "disc_model",
    "method",
    "K_vectors",
    "validity",
    "prob_plausibility",
    "cf_belongs_to_group",
    "log_density_cf",
    "proximity_continuous_manhattan",
    "proximity_continuous_euclidean",
    "isolation_forest_scores_cf",
    "lof_scores_cf",
    "time",
]
datasets = [
    "BlobsDataset",
    "LawDataset",
    "MoonsDataset",
    "WineDataset",
    "HelocDataset",
    "DigitsDataset",
]
disc_models = ["MultinomialLogisticRegression", "MultilayerPerceptron"]
global_methods = ["AReS", "GLOBE_CE", "RPPCEF_GLOBAL"]  # "GCE"
local_methods = ["wach", "Artelt", "RPPCEF_LOCAL"]  # "PPCEF_2"
group_methods = ["RPPCEF"]  # "GLANCE",
methods = global_methods + local_methods + group_methods

dataset = datasets[5]
disc_model = disc_models[0]
# method = local_methods[2]

df_all = pd.DataFrame(columns=columns)
for disc_model in disc_models:
    for dataset in datasets:
        for method in group_methods:
            df = get_results(dataset, disc_model, method, columns)
            df_all = pd.concat([df_all, df], axis=0)

BlobsDataset MultinomialLogisticRegression RPPCEF
LawDataset MultinomialLogisticRegression RPPCEF
MoonsDataset MultinomialLogisticRegression RPPCEF
WineDataset MultinomialLogisticRegression RPPCEF
HelocDataset MultinomialLogisticRegression RPPCEF
DigitsDataset MultinomialLogisticRegression RPPCEF
BlobsDataset MultilayerPerceptron RPPCEF
LawDataset MultilayerPerceptron RPPCEF
MoonsDataset MultilayerPerceptron RPPCEF
WineDataset MultilayerPerceptron RPPCEF
HelocDataset MultilayerPerceptron RPPCEF
DigitsDataset MultilayerPerceptron RPPCEF


In [12]:
df_all.groupby(["disc_model", "dataset", "method"]).mean().round(2)

K_vectors  validity  \
disc_model                    dataset       method                       
MultilayerPerceptron          BlobsDataset  RPPCEF       1.0      1.00   
                              DigitsDataset RPPCEF       4.0      1.00   
                              HelocDataset  RPPCEF      10.0      0.98   
                              LawDataset    RPPCEF       2.0      1.00   
                              MoonsDataset  RPPCEF       2.4      1.00   
                              WineDataset   RPPCEF       1.4      1.00   
MultinomialLogisticRegression BlobsDataset  RPPCEF       1.0      1.00   
                              DigitsDataset RPPCEF       4.0      1.00   
                              HelocDataset  RPPCEF      10.0      1.00   
                              LawDataset    RPPCEF       1.2      1.00   
                              MoonsDataset  RPPCEF       2.0      1.00   
                              WineDataset   RPPCEF       1.2      1.00   

                                                    prob_plausibility  \
disc_model                    dataset       method                      
MultilayerPerceptron          BlobsDataset  RPPCEF               0.92   
                              DigitsDataset RPPCEF               0.83   
                              HelocDataset  RPPCEF               0.18   
                              LawDataset    RPPCEF               0.85   
                              MoonsDataset  RPPCEF               0.92   
                              WineDataset   RPPCEF               1.00   
MultinomialLogisticRegression BlobsDataset  RPPCEF               0.92   
                              DigitsDataset RPPCEF               0.85   
                              HelocDataset  RPPCEF               0.16   
                              LawDataset    RPPCEF               0.83   
                              MoonsDataset  RPPCEF               0.77   
                              WineDataset   RPPCEF               1.00   

                                                    cf_belongs_to_group  \
disc_model                    dataset       method                        
MultilayerPerceptron          BlobsDataset  RPPCEF                 1.00   
                              DigitsDataset RPPCEF                 0.85   
                              HelocDataset  RPPCEF                 0.99   
                              LawDataset    RPPCEF                 0.97   
                              MoonsDataset  RPPCEF                 0.88   
                              WineDataset   RPPCEF                 0.98   
MultinomialLogisticRegression BlobsDataset  RPPCEF                 1.00   
                              DigitsDataset RPPCEF                 0.85   
                              HelocDataset  RPPCEF                 0.99   
                              LawDataset    RPPCEF                 0.99   
                              MoonsDataset  RPPCEF                 0.99   
                              WineDataset   RPPCEF                 1.00   

                                                    log_density_cf  \
disc_model                    dataset       method                   
MultilayerPerceptron          BlobsDataset  RPPCEF            2.88   
                              DigitsDataset RPPCEF          -99.00   
                              HelocDataset  RPPCEF           14.96   
                              LawDataset    RPPCEF            1.70   
                              MoonsDataset  RPPCEF            1.67   
                              WineDataset   RPPCEF            7.86   
MultinomialLogisticRegression BlobsDataset  RPPCEF            2.86   
                              DigitsDataset RPPCEF          -99.04   
                              HelocDataset  RPPCEF           18.52   
                              LawDataset    RPPCEF            1.67   
                              MoonsDataset  RPPCEF            1.24   
                              

In [13]:
df_all.to_csv("results_GROUP.csv", index=False)